# DHTML

In [ ]:
# 동적HTML(DHTML): 브라우저에 있는 DOM  // 네트워크.raw HTML 차이, WHY WHEN HOW?
# 일반적인 환경
# Client                  Server
#         ----(req)----->
#         <----(HTML)----
#         -(Form POST)-->
# (Reload)<----(HTML)----

# DHTML
# Client                  Server
#          ---(req)---->
#  (DOM)   <---(HTML)---
#          ---(AJAX)--->
#          <---(JSON)---

In [69]:
from requests import request
from requests.compat import urljoin, urlencode

In [2]:
url = 'https://pythonscraping.com/pages/javascript/ajaxDemo.html'
resp = request('get', url)
# status_code, reason, header
resp.status_code, resp.reason, resp.headers

(200,
 'OK',
 {'Server': 'nginx', 'Date': 'Tue, 18 Mar 2025 00:16:51 GMT', 'Content-Type': 'text/html', 'Content-Length': '348', 'Connection': 'keep-alive', 'X-Accel-Version': '0.01', 'Last-Modified': 'Sat, 09 Jun 2018 19:16:02 GMT', 'ETag': '"208-56e3a58e34080-gzip"', 'Accept-Ranges': 'bytes', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'X-Powered-By': 'PleskLin'})

In [3]:
from bs4 import BeautifulSoup

In [6]:
dom = BeautifulSoup(resp.text, 'html.parser')
dom.body # 주소가 동일하나, 시간이 지남에 따라 내용이 바뀜(javascript)

<body>
<div id="content">
This is some content that will appear on the page while it's loading. You don't care about scraping this.
</div>
<script>
$.ajax({
    type: "GET",
    url: "loadedContent.php",
    success: function(response){

	setTimeout(function() {
	    $('#content').html(response);
	}, 2000);
    }
  });

function ajax_delay(str){
 setTimeout("str",2000);
}
</script>
</body>

In [7]:
dom.select_one('#content')

<div id="content">
This is some content that will appear on the page while it's loading. You don't care about scraping this.
</div>

In [9]:
import re

ajax = re.search(r'url:\s*"(.+?)"', dom.body.script.contents[0]).group(1)
nurl = urljoin(url, ajax) # 데이터를 제공해주는 새로운 사이트 주소

In [10]:
resp = request('get', nurl)
resp.status_code, resp.reason, resp.headers

(200,
 'OK',
 {'Server': 'nginx', 'Date': 'Tue, 18 Mar 2025 00:24:33 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '106', 'Connection': 'keep-alive', 'X-Powered-By': 'PHP/7.4.33, PleskLin', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip'})

In [12]:
resp.text # 2초 로딩 후 생성되는 중요 텍스트

'Here is some important text you want to retrieve! <p/><button id="loadedButton">A button to click!</button>'

In [13]:
dom.head # jquery -> script -> loadedcontents(xhr)에 요청

<head>
<title>Some JavaScript-loaded content</title>
<script src="../js/jquery-2.1.1.min.js"></script>
</head>

Here is some important text you want to retrieve! <p/>
<button id="loadedButton">A button to click!</button>



```html
...
<script>
$.ajax({
    type: "GET",
    url: "loadedContent.php",
    success: function(response){

	setTimeout(function() {
	    $('#content').html(response);
	}, 2000);
    }
  });

function ajax_delay(str){
 setTimeout("str",2000);
}
</script>
...
```




```html
Here is some important text you want to retrieve! <p/>
<button id="loadedButton">A button to click!</button>
```



In [16]:
url = 'https://brunch.co.kr/search?'
params = {
    'q': '카리나',
    'type': 'article'
}
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Whale/3.26.244.21 Safari/537.36'
}

resp = request('get', url, params=params, headers=headers)
resp.status_code, resp.reason, resp.headers

(200,
 '',
 {'Date': 'Tue, 18 Mar 2025 00:33:25 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Content-Language': 'en-US', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Content-Encoding': 'gzip'})

In [17]:
dom = BeautifulSoup(resp.text, 'html.parser')
dom.main

<main>
<button class="btn_close_side_menu img_side_menu ico_close f_r text_hide" id="btnCloseSideMenu" type="button">닫기</button>
<div class="wrap_side_profile logout">
<div class="logo_service"></div>
<p class="slogan">You can make anything<br/>by writing</p>
<p class="slogan_writer">C.S.Lewis</p>
<a class="#side_request btn_apply_author search" data-tiara-action-name="메뉴 &gt; 시작하기 버튼 클릭" data-tiara-layer="login" href="#" id="brunchSigninButton"><button class="btn_request">브런치스토리 시작하기</button></a>
</div>
<div class="wrap_side_service_menu">
<ul>
<li>
<a class="menu_word7 #side_brunch" data-tiara-action-name="메뉴 &gt; 브런치 홈 버튼 클릭" data-tiara-layer="top" href="/"><span class="bar_left"></span>브런치스토리 홈<span class="bar_right"></span></a>
</li>
<li>
<a class="menu_word8 #side_now brunchNowLink" data-tiara-action-name="메뉴 &gt; 브런치 나우 버튼 클릭" data-tiara-layer="now" href="/now"><span class="bar_left"></span>브런치스토리 나우<span class="bar_right"></span></a>
</li>
<li>
<a class="menu_word8 #side_publis

In [19]:
dom.select_one('#resultArticle')

<div class="search_result" id="resultArticle"></div>

In [20]:
dom.select('script')

[<script src="//t1.daumcdn.net/brunch/static/libraries/js/jquery.min-3.6.4.js" type="text/javascript"></script>,
 <script src="//t1.daumcdn.net/brunch/static/libraries/js/jquery.cookie.min-1.4.1.js" type="text/javascript"></script>,
 <script src="//t1.daumcdn.net/brunch/static/libraries/js/jquery.xdomainrequest.min-1.0.4.js" type="text/javascript"></script>,
 <script src="//t1.daumcdn.net/brunch/static/libraries/js/jquery.word-break-keep-all.min-1.3.0.js" type="text/javascript"></script>,
 <script src="//t1.daumcdn.net/brunch/static/libraries/js/handlebars.runtime.min-v4.7.8.js" type="text/javascript"></script>,
 <script src="//t1.daumcdn.net/brunch/static/libraries/js/sentry.min.5.27.4.js?v=20221124" type="text/javascript"></script>,
 <script src="//t1.daumcdn.net/brunch/static/libraries/js/clipboard.min.js" type="text/javascript"></script>,
 <script src="//t1.daumcdn.net/tiara/js/v1/tiara.min.js" type="text/javascript"></script>,
 <script>
     window.Guard = {
         hasError:  fa

### Brunch에서 제목 가져오기

In [22]:
url = 'https://api.brunch.co.kr/v1/search/article'

# 페이지 번호, 페이지 당 글 개수, bold 하이라이트, escape, order(정확도순, 최신은 recency)
params = {
    'q': '카리나',
    'page': '1',
    'pageSize': '20',
    'highlighter': 'y',
    'escape': 'y',
    'sortBy': 'accu'
}

resp = request('get', url, params=params, headers=headers)
resp.status_code, resp.reason, resp.headers['content-type']

(200, '', 'application/json;charset=UTF-8')

In [23]:
resp.request.url

'https://api.brunch.co.kr/v1/search/article?q=%EC%B9%B4%EB%A6%AC%EB%82%98&page=1&pageSize=20&highlighter=y&escape=y&sortBy=accu'

In [26]:
result = resp.json()
result.keys()

dict_keys(['desc', 'code', 'title', 'extra', 'data', 'version', 'redirectUrl', 'fallback'])

In [24]:
resp.json()['data']['list'][0].keys()

dict_keys(['id', 'version', 'no', 'userId', 'userName', 'status', 'magazineNo', 'magazineAddress', 'magazineAddressUri', 'magazineArticleId', 'magazineTitle', 'magazineStatus', 'magazineType', 'brunchbookSerialStatus', 'magazineCoverImage', 'commentWritable', 'commentCount', 'donationCommentCount', 'donationCommentNoJson', 'donationCommentList', 'donationTotalAmount', 'likeCount', 'title', 'subTitle', 'contentSummary', 'contentHash', 'authorComment', 'membershipContent', 'promotionEnabled', 'createTime', 'updateTime', 'publishTime', 'publishTimestamp', 'restrictTime', 'content', 'profileId', 'coverInfo', 'articleImageList', 'socialShareTotalCount', 'coverColor', 'articleKeywords', 'readSeconds', 'fromTime', 'toTime', 'orderBy', 'chapterNo', 'seq', 'tempBrunchbookNo', 'tempBrunchbookTitle', 'tempBrunchbookStatus', 'tempBrunchbookAddress', 'readTime', 'flattenedSeq', 'viewCount', 'kakaoShareCount', 'kakaoTalkShareCount', 'facebookShareCount', 'twitterShareCount', 'contentIdParam', 'homeD

In [28]:
result['data']['list'][0]

{'id': None,
 'version': None,
 'no': 137,
 'userId': 'eL95',
 'userName': None,
 'status': 'publish',
 'magazineNo': 0,
 'magazineAddress': None,
 'magazineAddressUri': None,
 'magazineArticleId': 0,
 'magazineTitle': None,
 'magazineStatus': None,
 'magazineType': None,
 'brunchbookSerialStatus': None,
 'magazineCoverImage': None,
 'commentWritable': False,
 'commentCount': 2,
 'donationCommentCount': 0,
 'donationCommentNoJson': None,
 'donationCommentList': None,
 'donationTotalAmount': 0,
 'likeCount': 0,
 'title': '<b>카리나</b>가 전하는 매운맛의 혁명! 스프라이트 광고 공개',
 'subTitle': None,
 'contentSummary': '<b>카리나</b>가 전하는 매운맛의 혁명! 스프라이트 광고 공개 &lt;사진=코카콜라&gt; 코카콜라가 세계 1위 사이다 브랜드 스프라이트의 새로운 광고 영상을 공개했습니다. 이번 광고는 걸그룹 에스파(aespa)의 리더 <b>카리나</b>와 손잡고 매운 음식을 더욱 특별하게 즐기는...',
 'contentHash': None,
 'authorComment': '작가의 한마디가 나옵니다.',
 'membershipContent': False,
 'promotionEnabled': False,
 'createTime': None,
 'updateTime': None,
 'publishTime': 1741094284000,
 'publishTimestamp': 1741094284000,
 'rest

In [30]:
for row in result['data']['list']:
  print(row['title'])

<b>카리나</b>가 전하는 매운맛의 혁명! 스프라이트 광고 공개
파스쿠찌 - <b>카리나</b>가 모든 걸 해결해주진 않는다
두 명의 <b>카리나</b>?! (ft. 파스쿠찌 x <b>카리나</b>)
<b>카리나</b>가 전하는 크리스마스
이재욱, 입대 후 <b>카리나</b>와 결별 후유증?
“<b>카리나</b>만큼 예쁘다”…SM 하츠투하츠, 2월24일 데뷔
<b>카리나</b> 붕어빵
새로운 산타가 <b>카리나</b>? 신선하다.
공효진 절대 안 하는데, <b>카리나</b>는 한다 &#34;건강에 최악&#34;
안나<b>카리나</b>
<b>카리나</b>
4세대 맥주 ‘크러시’- <b>카리나</b>로 매출 46% up
안나<b>카리나</b> 장미 &amp; 왁스핑크
&#39;주술회전&#39; 열풍, 에스파 <b>카리나</b>가 좋아한다고?
유지민 vs <b>카리나</b>
&#39;<b>카리나</b>의 굴욕&#39;과 군중심리
<b>카리나</b>, 이강인, 류준열 그리고 대중
<b>카리나</b> 연애 무엇이 문제인가


In [32]:
# 1번부터 5번 페이지까지 제목 가져오기
for i in range(1,6):
  params['page'] = i
  resp = request('get', url, params=params, headers=headers)
  if re.search(r'application/json', resp.headers['content-type']):
    result = resp.json()
    # if 'data' in resp.join()
    # if 'list' in resp.join()['data']
    for row in result['data']['list']:
      print(row['title'])

<b>카리나</b>가 전하는 매운맛의 혁명! 스프라이트 광고 공개
파스쿠찌 - <b>카리나</b>가 모든 걸 해결해주진 않는다
두 명의 <b>카리나</b>?! (ft. 파스쿠찌 x <b>카리나</b>)
<b>카리나</b>가 전하는 크리스마스
이재욱, 입대 후 <b>카리나</b>와 결별 후유증?
“<b>카리나</b>만큼 예쁘다”…SM 하츠투하츠, 2월24일 데뷔
<b>카리나</b> 붕어빵
새로운 산타가 <b>카리나</b>? 신선하다.
공효진 절대 안 하는데, <b>카리나</b>는 한다 &#34;건강에 최악&#34;
안나<b>카리나</b>
<b>카리나</b>
4세대 맥주 ‘크러시’- <b>카리나</b>로 매출 46% up
안나<b>카리나</b> 장미 &amp; 왁스핑크
&#39;주술회전&#39; 열풍, 에스파 <b>카리나</b>가 좋아한다고?
유지민 vs <b>카리나</b>
&#39;<b>카리나</b>의 굴욕&#39;과 군중심리
<b>카리나</b>, 이강인, 류준열 그리고 대중
<b>카리나</b> 연애 무엇이 문제인가
&#34;결국 열애설 사과문 올렸다&#34; <b>카리나</b>, 해외 팬들→비난
<b>카리나</b> 열애설... 손가락질 받아야 할 대상은?
&#34;어떻게든 만났을 운명&#34; 이재욱, ♥<b>카리나</b>와 여행예능?
이재욱 <b>카리나</b> 열애설에 &#39;환승연애설&#39;까지
차은우는 왜 차은우이고 <b>카리나</b>는 왜 <b>카리나</b>일까?
[<b>카리나</b>]의 출연을 상상하며 만들어 보는 캐릭터
누구나 설득되는 퇴사사유 3가지
여전히 종이에 필기하시는 분 없나요
<b>카리나</b> 3행시 할게요 #에스파 #aespa
<b>카리나</b> 3행시 할게요 #에스파 #aespa
<b>카리나</b>가 선택한 공항패션 - Weekly Celeb
목도리가 지어줄 작은 집
이것도 퇴사 사유가 되나요? (1) 거리
주씨, 후레쉬 11
남자들이 쉽게 꼬이는 여자들의 &#39;공통된 행동&#39; 7가지
in Drop or Go] 3월 1주

In [33]:
url = 'https://brunch.co.kr'
resp = request('get', url, headers=headers)
dom = BeautifulSoup(resp.text, 'html.parser')

In [35]:
for el in dom.select('form input[name]'): # key:value
  print(el.attrs['name'], el.attrs['value'])

q 
profileId 


In [36]:
params['q']

'카리나'

In [39]:
dom.select_one('form').attrs

{'action': '/search', 'method': 'get'}

In [40]:
url = 'https://api.brunch.co.kr/v1/search/article'

In [41]:
url = 'https://brunch.co.kr/search'
resp = request('get', url, headers=headers)
dom = BeautifulSoup(resp.text, 'html.parser')

In [42]:
for el in dom.select('form > input[name]'):
  print(el.attrs['name'], el.attrs['value'])

q 
profileId 
q 
type article


In [43]:
dom.select_one('form').attrs
# Client          /search           /api
# [form] ---------> GET
# <input name='' value=''> => ?name=value
#                   JS   ----------> GET
#                           AJAX
#                        <---------- JSON
# DHTML <----------

{'action': '/search', 'method': 'get'}

In [ ]:
# HTML ---(req)--->
#     FORM  action => 주소
#     INPUT name, value => ?name=value
# <script src=".js">
# <script>
#   $.ajax();
# </script> => JS 해석X, 브라우저(개발자도구)-네트워크탭

# JS(XHR) ---(req)---> Server
#         <----------- content-type
# Data(DOM을 새롭게 그리지 않는다) => 해석

### Brunch에서 검색창에 입력할 때마다 추천검색어 가져오기

In [79]:
url = 'https://api.brunch.co.kr/v1/search/live'
params = { 'q': None }
params['q'] = '카리나'
resp = request('get', url, params=params, headers=headers)

In [80]:
for a in resp.json()['data']['article']:
  print(a['title'])

# 'magazine' in resp.json()['data'].keys()
# 'article'
# filter(lambda k:re.match('magazine|article'), resp.json()['data'].keys())

<b>카리나</b>가 전하는 매운맛의 혁명! 스프라이트 광고 공개
<b>카리나</b>
파스쿠찌 - <b>카리나</b>가 모든 걸 해결해주진 않는다
두 명의 <b>카리나</b>?! (ft. 파스쿠찌 x <b>카리나</b>)
<b>카리나</b>가 전하는 크리스마스
이재욱, 입대 후 <b>카리나</b>와 결별 후유증?
“<b>카리나</b>만큼 예쁘다”…SM 하츠투하츠, 2월24일 데뷔


### 네이버 검색엔진에서 추천검색어 가져오기

In [67]:
url = 'https://ac.search.naver.com/nx/ac'

params = {
    'q': None,
    'con':'0',
    'frm':'nv',
    'ans':'2',
    'r_format':'json',
    'r_enc':'UTF-8',
    'r_unicode':'0',
    't_koreng':'1',
    'run':'2',
    'rev':'4',
    'q_enc':'UTF-8',
    'st':'100'
}
# Client              Server
#       ----req------>
#       <----resp-----

# $.ajax({success:func(response) 어떻게...})
# ?callback=_jsonp_3
# func callback1() ...
# func callback2() ...
# func callback3() <--

In [68]:
while True:
  query = input()
  if query == 'q':
    break

  params['q'] = query
  resp = request('GET', url, params=params, headers=headers)
  if re.search(r'application/json', resp.headers['content-type']):
    print(resp.json()['items'])

카
[[['카카오페이지', '0'], ['카카오', '2'], ['카카오웹툰', '0'], ['카페', '0'], ['카리나', '0'], ['카피바라', '0'], ['카카오뱅크', '0'], ['카카오톡', '0'], ['카카오맵', '0'], ['카니발', '0']]]
카리
[[['카리나', '0'], ['카리토포텐', '0'], ['카리나 단발', '0'], ['카리나 이재욱', '0'], ['카린', '0'], ['카리나 지디', '0'], ['카링킷', '0'], ['카리나 과사', '0'], ['카리나 중단발', '0'], ['카리스마', '0']]]
카리나
[[['카리나', '0'], ['카리나 이재욱', '0'], ['카리나 중단발', '0'], ['카리나 단발', '0'], ['카리나 배경화면', '0'], ['카리나 셀카', '0'], ['카리나 본명', '0'], ['카리나 사과문', '0'], ['카리나 up', '0'], ['카리나 워터밤', '0']]]
q


## 네이버웹툰 이미지 수집기

### 크롤러를 이용한 **수집기**

url 수집을 우선적으로 한 후 이미지를 저장함

1단계: 네이버웹툰 메인 페이지
```
[WEB] : 'https://comic.naver.com/index' ==> DOM1(div#root)
[AJAX] :'https://comic.naver.com/api/home/component?type=DAILY_WEBTOON&order=USER'
        titleList:
          0: { "titleId": 783053, "titleName": "김부장",... }
```


2단계: 김부장 내 회차별 목록
```
[WEB] : 'https://comic.naver.com/webtoon/list?titleId=783053' ==> DOM2(div#root)
[AJAX] :'https://comic.naver.com/api/article/list?titleId=783053&page=1&sort=DESC'
        articleList:
          0: { no:177,... }
```


3단계: 특정 회차 접근
```
[WEB] : 'https://comic.naver.com/webtoon/detail?titleId=783053&no=177&week=tue' ==> DOM3(#sectionContWide)
```

4단계: 웹툰 내 이미지 저장
```
[IMG] : url -> 저장
```

In [75]:
URLs = ['https://comic.naver.com/api/home/component?type=DAILY_WEBTOON&order=USER']
Seens = []

while URLs:
    seed = URLs.pop(0)
    Seens.append(seed)

    resp = request('GET', seed, headers=headers)
    if resp.status_code != 200:
        continue

    # 4단계: 추출한 이미지 다운로드
    if re.search(r'image/jpeg', resp.headers['content-type']):
        file = re.search(r'.+/(.+[.]jpg)$', seed).group(1)
        with open(file, 'wb') as fp:
            fp.write(resp.content)

    # 3단계: 특정 회차에서 이미지 가져오기
    if re.search(r'text/html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html.parser')
        for img in dom.select('#sectionContWide img[src]'):
            nurl = img.attrs['src']
            if nurl not in URLs and nurl not in Seens:
                URLs.append(nurl)

    if not re.search(r'application/json', resp.headers['content-type']):
        continue

    result = resp.json()

    # 1단계: 웹툰별 목록
    if 'titleList' in result.keys():
        for r in result['titleList']:
            url = 'https://comic.naver.com/api/article/list'
            params = {
                'titleId':r['titleId'],
                'page':'1',
                'sort':'DESC'
            }
            nurl = url+'?'+urlencode(params)
            if nurl not in URLs and nurl not in Seens:
                URLs.append(nurl)
                break

    # 2단계: 특정 웹툰의 회차 목록
    if 'articleList' in result.keys():
        for r in result['articleList']:
            url = 'https://comic.naver.com/webtoon/detail'
            params = {
                'titleId':re.search(r'titleId=(\d+)&', seed).group(1),
                'no':r['no'],
                'week':'tue'
            }
            nurl = url+'?'+urlencode(params)
            if nurl not in URLs and nurl not in Seens:
                URLs.append(nurl)
                break

In [76]:
URLs

[]

In [77]:
Seens

['https://comic.naver.com/api/home/component?type=DAILY_WEBTOON&order=USER',
 'https://comic.naver.com/api/article/list?titleId=783053&page=1&sort=DESC',
 'https://comic.naver.com/webtoon/detail?titleId=783053&no=177&week=tue',
 'https://image-comic.pstatic.net/webtoon/783053/177/20250211203628_3c02b34fc5e5dd07241cff64d232bdaf_IMAG01_1.jpg',
 'https://image-comic.pstatic.net/webtoon/783053/177/20250211203628_3c02b34fc5e5dd07241cff64d232bdaf_IMAG01_2.jpg',
 'https://image-comic.pstatic.net/webtoon/783053/177/20250211203628_3c02b34fc5e5dd07241cff64d232bdaf_IMAG01_3.jpg',
 'https://image-comic.pstatic.net/webtoon/783053/177/20250211203628_3c02b34fc5e5dd07241cff64d232bdaf_IMAG01_4.jpg',
 'https://image-comic.pstatic.net/webtoon/783053/177/20250211203628_3c02b34fc5e5dd07241cff64d232bdaf_IMAG01_5.jpg',
 'https://image-comic.pstatic.net/webtoon/783053/177/20250211203628_3c02b34fc5e5dd07241cff64d232bdaf_IMAG01_6.jpg',
 'https://image-comic.pstatic.net/webtoon/783053/177/20250211203628_3c02b34f

### 스크래퍼를 이용한 수집기

In [88]:
url = 'https://comic.naver.com/api/home/component?type=DAILY_WEBTOON&order=USER'

resp = request('GET', url, headers=headers)
resp.status_code, resp.reason, resp.headers

(200,
 'OK',
 {'Content-Type': 'application/json', 'x-content-type-options': 'nosniff', 'x-xss-protection': '1; mode=block', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'x-frame-options': 'SAMEORIGIN', 'Content-Encoding': 'gzip', 'referrer-policy': 'unsafe-url', 'Server': 'nfront', 'Content-Length': '9148', 'Date': 'Tue, 18 Mar 2025 02:37:26 GMT', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Set-Cookie': 'XSRF-TOKEN=6ae492fb-d526-472b-a682-2809511b13dd; Path=/'})

In [89]:
webtoonList = resp.json()

for webtoon in webtoonList['titleList']:
  print(webtoon['titleId'], webtoon['titleName'])

783053 김부장
821597 괴력 난신
835736 중증외상센터 : 외과의사 백강혁
828715 절대회귀
833255 낢이 사는 이야기 - 계속되는 미미한 인생
819929 서울 자가에 대기업 다니는 김 부장 이야기
812594 신입사원 김철수
813454 이섭의 연애
814543 마음의소리2
789979 멸망 이후의 세계
730656 사신소년
822556 유부 감자
743139 한림체육관
774358 천마는 평범하게 살 수 없다
833705 귀혼
836848 44교시 생존수업
801711 시한부인 줄 알았어요!
773459 용사가 돌아왔다
817247 못 잡아먹어서 안달
783877 하북팽가 막내아들
834369 엄마를 만나러 가는 길
824750 서툰 군인
818448 욕망일기Deep
833244 티키타카
804653 유사연애
832557 회귀한 용병은 다 계획이 있다
795658 초인의 게임
817366 내 남편의 정부에게
800726 사표내고 이계에서 힐링합니다
828920 청춘 러브썸
833416 구룡:사로카
804783 사변괴담
809618 신과함께 돌아온 기사왕님
812161 도태교실
814535 백씨세가 시한부 공자
828908 긁지 않은 복권
698918 원주민 공포만화
829743 아포칼립스에 물류센터를 숨김
830106 백작가의 비밀스런 시녀님
833243 전리품 공작부인
822198 사람의 탈
818434 아카데미 플레이어를 죽였다
818969 약 파는 황태자
818192 염라강림
835751 윈터브리즈
828292 개집사
836361 킬링필드
616239 윌유메리미
814834 체탐자
836239 2025 졸업작품.zip
828402 천만청춘
765156 빌런투킬
752532 연우의 순정
823434 나만 보는 탑 공략집
830053 가공낙원
826887 죽어 천국에 가다
833873 야만의 청혼
831168 귀공자
835582 인생내꺼
819768 맹수 사용 설명서
654138 은주의 방 2~3부
822931 불 빌려드릴까요?
82

삼중 for문을 사용했기 때문에 시간복잡도가 좋지 않다. crawling과는 link를 수집하는 측면에서 다르다.

In [87]:
for title in webtoonList['titleList']:
  url = 'https://comic.naver.com/api/article/list?titleId='+title['titleId']+'&page=1&sort=DESC'
  resp = request('GET', url, headers=headers)
  webtoon = resp.json()
  for article in webtoon['articleList']:
    print(article['title'])

TypeError: can only concatenate str (not "int") to str